Test bodo snowflake connectivity

In [1]:
import snowflake.connector
#import bodo
import time
from sqlalchemy.dialects import registry
registry.register('snowflake', 'snowflake.sqlalchemy', 'dialect')

In [2]:
import os
with open("./snowflake-secrets.txt","r") as f:
    for line in f:
        args=line.split("=")
        os.environ[args[0]]=args[1].strip()
username=os.environ["username"]
password=os.environ["password"]
account=os.environ["account"]
warehouse="BODO_VW"
database="BODO_DB"
role="BODO_R1"

In [3]:
import bodo
import pandas as pd
@bodo.jit()
def test_connection():
    df=pd.read_sql('select * from PUBLIC.TRIPS limit 200', f"snowflake://{username}:{password}@{account}/{database}/public?role={role}&warehouse={warehouse}",)
    print(len(df))
    return df
df=test_connection()
df

200


,tripduration,starttime,stoptime,start_station_id,start_station_name,start_station_latitude,start_station_longitude,end_station_id,end_station_name,end_station_latitude,end_station_longitude,bikeid,membership_type,usertype,birth_year,gender
0,204,2015-05-13 07:02:53,2015-05-13 07:06:17,449,W 52 St & 9 Ave,40.764618,-73.987895,529,W 42 St & 8 Ave,40.757570,-73.990985,17300,NaN,Subscriber,1984,1
1,154,2015-05-13 07:02:55,2015-05-13 07:05:29,164,E 47 St & 2 Ave,40.753231,-73.970325,367,E 53 St & Lexington Ave,40.758281,-73.970694,20578,NaN,Subscriber,1982,1
2,336,2015-05-13 07:02:55,2015-05-13 07:08:32,479,9 Ave & W 45 St,40.760193,-73.991255,490,8 Ave & W 33 St,40.751551,-73.993934,19530,NaN,Subscriber,1983,2
3,940,2015-05-13 07:03:01,2015-05-13 07:18:41,449,W 52 St & 9 Ave,40.764618,-73.987895,491,E 24 St & Park Ave S,40.740964,-73.986022,15481,NaN,Subscriber,1955,2
4,1158,2015-05-13 07:03:01,2015-05-13 07:22:20,482,W 15 St & 7 Ave,40.739355,-73.999318,500,Broadway & W 51 St,40.762288,-73.983362,18608,NaN,Subscriber,1978,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,842,2015-08-05 17:26:43,2015-08-05 17:40:46,448,W 37 St & 10 Ave,40.756604,-73.997901,238,Bank St & Washington St,40.736197,-74.008592,22069,NaN,Subscriber,1983,2
196,477,2015-08-05 17:26:48,2015-08-05 17:34:46,236,St Marks Pl & 2 Ave,40.728419,-73.987140,268,Howard St & Centre St,40.719105,-73.999733,16133,NaN,Subscriber,1991,2
197,263,2015-08-05 17:26:48,2015-08-05 17:31:12,427,Bus Slip & State St,40.701907,-74.013942,363,West Thames St,40.708347,-74.017134,19266,NaN,Subscriber,1982,1
198,930,2015-08-05 17:26:49,2015-08-05 17:42:19,248,Laight St & Hudson St,40.721854,-74.007718,2003,1 Ave & E 18 St,40.733812,-73.980544,22901,NaN,Subscriber,1987,1


In [4]:
password="badpass1"

In [5]:
import bodo
import pandas as pd
@bodo.jit()
def test_connection():
    df=pd.read_sql('select * from PUBLIC.WEATHER', f"snowflake://{username}:{password}@{account}/{database}/public?role={role}&warehouse={warehouse}",)
    return df
try:
    df=test_connection()
except Exception as e:
    print(f"error connecting to database {e}")

error connecting to database pd.read_sql(): Error executing query `select * from PUBLIC.WEATHER`.
DatabaseError:'250001 (08001): Failed to connect to DB: persistent_partner.us-east-1.snowflakecomputing.com:443. Incorrect username or password was specified.'


In [ ]:
import time
import snowflake.connector
def snowtest():
    t0 = time.time()
    print("start reading")
    cursor = snowflake.connector.connect(
        user=username,
        account=account,
        password=password,
        warehouse=warehouse,
        database=database,
        schema=schema
    ).cursor()
    sql = f'select * from {schema}.{table} limit {limit}'
    cursor.execute(sql)
    df = cursor.fetch_pandas_all()
    #df = df.groupby("REF").sum()
    print(f"df read time connector: {round(time.time() - t0, 3)}")
    return df
df=snowtest()
df